In [2]:
pip install python-docx
pip install beautifulsoup4



   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/239.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/239.6 kB ? eta -:--:--
   ----- --------------------------------- 30.7/239.6 kB 145.2 kB/s eta 0:00:02
   ----- --------------------------------- 30.7/239.6 kB 145.2 kB/s eta 0:00:02
   ---------- ---------------------------- 61.4/239.6 kB 204.8 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/239.6 kB 261.7 kB/s eta 0:00:01
   ---------------------- --------------- 143.4/239.6 kB 387.0 kB/s eta 0:00:01
   -------------------------------------  235.5/239.6 kB 554.9 kB/s eta 0:00:01
   -------------------------------------  235.5/239.6 kB 554.9 kB/s eta 0:00:01
   -------------------------------------- 239.6/239.6 kB 473.9 kB/s eta 0:00:00


In [5]:
import requests
from docx import Document
from bs4 import BeautifulSoup
from urllib.parse import urlparse

def html_to_docx(input_path, output_file):
    if urlparse(input_path).scheme in ('http', 'https'):
        # Fetch the HTML content from the URL
        response = requests.get(input_path)
        html_content = response.text
    else:
        # Read the HTML content from the file
        with open(input_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Create a new Word document
    doc = Document()

    # Find all paragraphs in the HTML and add them to the Word document
    for paragraph in soup.find_all('p'):
        doc_paragraph = doc.add_paragraph()
        for elem in paragraph.contents:
            if elem.name == 'span':
                run = doc_paragraph.add_run()
                if 'style' in elem.attrs:
                    style = elem['style']
                    if 'color' in style:
                        color = style.split('color:', 1)[1].split(';')[0]
                        run.font.color.rgb = hex_to_rgb(color)
                    if 'font-weight:bold' in style:
                        run.bold = True
                    if 'font-style:italic' in style:
                        run.italic = True
                    if 'text-decoration:underline' in style:
                        run.underline = True
                run.text = elem.get_text()
            else:
                doc_paragraph.add_run(elem.get_text())

    # Save the Word document
    doc.save(output_file)

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Example usage
#html_to_docx('input.html', 'output.docx')  # For local file
html_to_docx('https://example.com/page.html', 'output.docx')  # For web URL
